In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

A = 1
omega = 2 * np.pi * 50
T = 1 / 50
t = np.linspace(0, 1, 1000)
phases = [0, np.pi/6, np.pi/4, np.pi/3, np.pi/2, 2*np.pi/3, 3*np.pi/4, 5*np.pi/6, np.pi,
          7*np.pi/6, 5*np.pi/4, 4*np.pi/3, 3*np.pi/2, 2*np.pi]
np.random.seed(0)  

def u(t):
    return np.heaviside(t, 1)

def pure_sine(t, phi, noise=False):
    signal = A * np.sin(omega * t + phi)
    if noise:
        signal += np.random.normal(0, 0.1, len(t))
    return signal

def sag(t, alpha, t1, t2, phi, noise=False):
    signal = (1 - alpha * (u(t - t1) - u(t - t2))) * np.sin(omega * t + phi)
    if noise:
        signal += np.random.normal(0, 0.1, len(t))
    return signal

def swell(t, alpha, t1, t2, phi, noise=False):
    signal = (1 + alpha * (u(t - t1) - u(t - t2))) * np.sin(omega * t + phi)
    if noise:
        signal += np.random.normal(0, 0.1, len(t))
    return signal

def interruption(t, alpha, t1, t2, phi, noise=False):
    signal = alpha * (u(t - t1) - u(t - t2)) * np.sin(omega * t + phi)
    if noise:
        signal += np.random.normal(0, 0.1, len(t))
    return signal

def harmonics(t, alpha1, alpha3, alpha5, alpha7, phi, noise=False):
    signal = alpha1 * np.sin(omega * t + phi) + alpha3 * np.sin(3 * omega * t + phi) + alpha5 * np.sin(5 * omega * t + phi) + alpha7 * np.sin(7 * omega * t + phi)
    if noise:
        signal += np.random.normal(0, 0.1, len(t))
    return signal

def oscillatory_transient(t, alpha, t1, tau, omega_n, phi, noise=False):
    signal = np.sin(omega * t + phi) + alpha * np.exp(-(t - t1) / tau) * np.sin(omega_n * (t - t1) + phi) * (u(t - t1) - u(t - t1 - 3 * tau))
    if noise:
        signal += np.random.normal(0, 0.1, len(t))
    return signal

def flicker(t, alpha_f, beta, phi, noise=False):
    signal = (1 + alpha_f * np.sin(beta * t)) * np.sin(omega * t + phi)
    if noise:
        signal += np.random.normal(0, 0.1, len(t))
    return signal

def impulse_transient(t, K, t1, t2, phi, noise=False):
    signal = np.sin(omega * t + phi) + np.sign(np.sin(omega * t + phi)) * K * (u(t - t1) - u(t - t2))
    if noise:
        signal += np.random.normal(0, 0.1, len(t))
    return signal

def notch(t, K, t1, t2, phi, noise=False):
    signal = np.sin(omega * t + phi) - np.sign(np.sin(omega * t + phi)) * K * (u(t - t1) - u(t - t2))
    if noise:
        signal += np.random.normal(0, 0.1, len(t))
    return signal

def sag_with_harmonics(t, alpha, t1, t2, alpha1, alpha3, alpha5, alpha7, phi, noise=False):
    signal = sag(t, alpha, t1, t2, phi) + harmonics(t, alpha1, alpha3, alpha5, alpha7, phi)
    if noise:
        signal += np.random.normal(0, 0.1, len(t))
    return signal

def swell_with_harmonics(t, alpha, t1, t2, alpha1, alpha3, alpha5, alpha7, phi, noise=False):
    signal = swell(t, alpha, t1, t2, phi) + harmonics(t, alpha1, alpha3, alpha5, alpha7, phi)
    if noise:
        signal += np.random.normal(0, 0.1, len(t))
    return signal

def sag_with_oscillation(t, alpha, phi, beta, t3, omega_n, noise=False):
    signal = A * np.sin(omega * t - phi) * (1 - alpha * (u(t - t3) - u(t - t3 - 3 * T))) + beta * np.exp(-(t - t3) / T) * np.sin(omega_n * (t - t3) + phi) * (u(t - t3) - u(t - t3 - 3 * T))
    if noise:
        signal += np.random.normal(0, 0.1, len(t))
    return signal

def swell_with_oscillation(t, alpha, phi, beta, t3, omega_n, noise=False):
    signal = A * np.sin(omega * t - phi) * (1 + alpha * (u(t - t3) - u(t - t3 - 3 * T))) + beta * np.exp(-(t - t3) / T) * np.sin(omega_n * (t - t3) + phi) * (u(t - t3) - u(t - t3 - 3 * T))
    if noise:
        signal += np.random.normal(0, 0.1, len(t))
    return signal

def swell_with_harmonics_oscillations(t, alpha, t1, t2, alpha1, alpha3, alpha5, alpha7, beta, phi, t3, omega_n, noise=False):
    signal = swell_with_harmonics(t, alpha, t1, t2, alpha1, alpha3, alpha5, alpha7, phi) + beta * np.exp(-(t - t3) / T) * np.sin(omega_n * (t - t3) + phi) * (u(t - t3) - u(t - t3 - 3 * T))
    if noise:
        signal += np.random.normal(0, 0.1, len(t))
    return signal

def sag_with_harmonics_oscillations(t, alpha, t1, t2, alpha1, alpha3, alpha5, alpha7, beta, phi, t3, omega_n, noise=False):
    signal = sag_with_harmonics(t, alpha, t1, t2, alpha1, alpha3, alpha5, alpha7, phi) + beta * np.exp(-(t - t3) / T) * np.sin(omega_n * (t - t3) + phi) * (u(t - t3) - u(t - t3 - 3 * T))
    if noise:
        signal += np.random.normal(0, 0.1, len(t))
    return signal

def plot_pq_event(event_func, t, save_path, *args):
    plt.figure(figsize=(10, 4))
    plt.plot(t, event_func(t, *args))
    plt.axis('off')
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    plt.close()

def generate_dataset():
    dataset = []
    labels = []
    noisy_count = {}
    
    for phi in phases:
        dataset.append((pure_sine(t, phi), 'PureSine', phi, False))
        dataset.append((pure_sine(t, phi, noise=True), 'PureSine', phi, True))
        noisy_count['PureSine'] = 2

    for alpha in np.linspace(0.1, 0.8, 5):
        for phi in phases:
            label = 'Sag'
            if noisy_count.get(label, 0) < 2:
                dataset.append((sag(t, alpha, 0.2, 0.5, phi, noise=True), label, phi, True))
                noisy_count[label] = noisy_count.get(label, 0) + 1
            dataset.append((sag(t, alpha, 0.2, 0.5, phi), label, phi, False))

    for alpha in np.linspace(0.1, 0.8, 5):
        for phi in phases:
            label = 'Swell'
            if noisy_count.get(label, 0) < 2:
                dataset.append((swell(t, alpha, 0.2, 0.5, phi, noise=True), label, phi, True))
                noisy_count[label] = noisy_count.get(label, 0) + 1
            dataset.append((swell(t, alpha, 0.2, 0.5, phi), label, phi, False))

    for alpha in np.linspace(0.1, 0.8, 5):
        for phi in phases:
            label = 'Interruption'
            if noisy_count.get(label, 0) < 2:
                dataset.append((interruption(t, alpha, 0.2, 0.5, phi, noise=True), label, phi, True))
                noisy_count[label] = noisy_count.get(label, 0) + 1
            dataset.append((interruption(t, alpha, 0.2, 0.5, phi), label, phi, False))

    for alpha1, alpha3, alpha5, alpha7 in zip(np.linspace(0.8, 1.2, 3), np.linspace(0.2, 0.5, 3), np.linspace(0.05, 0.1, 3), np.linspace(0.01, 0.05, 3)):
        for phi in phases:
            label = 'Harmonics'
            if noisy_count.get(label, 0) < 2:
                dataset.append((harmonics(t, alpha1, alpha3, alpha5, alpha7, phi, noise=True), label, phi, True))
                noisy_count[label] = noisy_count.get(label, 0) + 1
            dataset.append((harmonics(t, alpha1, alpha3, alpha5, alpha7, phi), label, phi, False))

    for alpha in np.linspace(0.1, 0.8, 5):
        for tau in np.linspace(0.01, 0.1, 5):
            for omega_n in np.linspace(2 * np.pi * 50, 2 * np.pi * 100, 5):
                for phi in phases:
                    label = 'OscillatoryTransient'
                    if noisy_count.get(label, 0) < 2:
                        dataset.append((oscillatory_transient(t, alpha, 0.2, tau, omega_n, phi, noise=True), label, phi, True))
                        noisy_count[label] = noisy_count.get(label, 0) + 1
                    dataset.append((oscillatory_transient(t, alpha, 0.2, tau, omega_n, phi), label, phi, False))

    for alpha_f, beta in zip(np.linspace(0.1, 0.4, 5), np.linspace(0.1, 0.4, 5)):
        for phi in phases:
            label = 'Flicker'
            if noisy_count.get(label, 0) < 2:
                dataset.append((flicker(t, alpha_f, beta, phi, noise=True), label, phi, True))
                noisy_count[label] = noisy_count.get(label, 0) + 1
            dataset.append((flicker(t, alpha_f, beta, phi), label, phi, False))

    for K in np.linspace(0.5, 1.5, 5):
        for phi in phases:
            label = 'ImpulseTransient'
            if noisy_count.get(label, 0) < 2:
                dataset.append((impulse_transient(t, K, 0.2, 0.25, phi, noise=True), label, phi, True))
                noisy_count[label] = noisy_count.get(label, 0) + 1
            dataset.append((impulse_transient(t, K, 0.2, 0.25, phi), label, phi, False))

    for K in np.linspace(0.5, 1.5, 5):
        for phi in phases:
            label = 'Notch'
            if noisy_count.get(label, 0) < 2:
                dataset.append((notch(t, K, 0.2, 0.25, phi, noise=True), label, phi, True))
                noisy_count[label] = noisy_count.get(label, 0) + 1
            dataset.append((notch(t, K, 0.2, 0.25, phi), label, phi, False))

    for alpha in np.linspace(0.1, 0.8, 5):
        for alpha1, alpha3, alpha5, alpha7 in zip(np.linspace(0.8, 1.2, 3), np.linspace(0.2, 0.5, 3), np.linspace(0.05, 0.1, 3), np.linspace(0.01, 0.05, 3)):
            for phi in phases:
                label = 'SagWithHarmonics'
                if noisy_count.get(label, 0) < 2:
                    dataset.append((sag_with_harmonics(t, alpha, 0.2, 0.5, alpha1, alpha3, alpha5, alpha7, phi, noise=True), label, phi, True))
                    noisy_count[label] = noisy_count.get(label, 0) + 1
                dataset.append((sag_with_harmonics(t, alpha, 0.2, 0.5, alpha1, alpha3, alpha5, alpha7, phi), label, phi, False))

    for alpha in np.linspace(0.1, 0.8, 5):
        for alpha1, alpha3, alpha5, alpha7 in zip(np.linspace(0.8, 1.2, 3), np.linspace(0.2, 0.5, 3), np.linspace(0.05, 0.1, 3), np.linspace(0.01, 0.05, 3)):
            for phi in phases:
                label = 'SwellWithHarmonics'
                if noisy_count.get(label, 0) < 2:
                    dataset.append((swell_with_harmonics(t, alpha, 0.2, 0.5, alpha1, alpha3, alpha5, alpha7, phi, noise=True), label, phi, True))
                    noisy_count[label] = noisy_count.get(label, 0) + 1
                dataset.append((swell_with_harmonics(t, alpha, 0.2, 0.5, alpha1, alpha3, alpha5, alpha7, phi), label, phi, False))

    for alpha in np.linspace(0.1, 0.8, 5):
        for beta in np.linspace(0.2, 0.5, 3):
            for omega_n in np.linspace(2 * np.pi * 50, 2 * np.pi * 100, 5):
                for phi in phases:
                    label = 'SagWithOscillation'
                    if noisy_count.get(label, 0) < 2:
                        dataset.append((sag_with_oscillation(t, alpha, phi, beta, 0.2, omega_n, noise=True), label, phi, True))
                        noisy_count[label] = noisy_count.get(label, 0) + 1
                    dataset.append((sag_with_oscillation(t, alpha, phi, beta, 0.2, omega_n), label, phi, False))

    for alpha in np.linspace(0.1, 0.8, 5):
        for beta in np.linspace(0.2, 0.5, 3):
            for omega_n in np.linspace(2 * np.pi * 50, 2 * np.pi * 100, 5):
                for phi in phases:
                    label = 'SwellWithOscillation'
                    if noisy_count.get(label, 0) < 2:
                        dataset.append((swell_with_oscillation(t, alpha, phi, beta, 0.2, omega_n, noise=True), label, phi, True))
                        noisy_count[label] = noisy_count.get(label, 0) + 1
                    dataset.append((swell_with_oscillation(t, alpha, phi, beta, 0.2, omega_n), label, phi, False))

    for alpha in np.linspace(0.1, 0.8, 5):
        for alpha1, alpha3, alpha5, alpha7 in zip(np.linspace(0.8, 1.2, 3), np.linspace(0.2, 0.5, 3), np.linspace(0.05, 0.1, 3), np.linspace(0.01, 0.05, 3)):
            for beta in np.linspace(0.2, 0.5, 3):
                for omega_n in np.linspace(2 * np.pi * 50, 2 * np.pi * 100, 5):
                    for phi in phases:
                        label = 'SwellWithHarmonicsOscillations'
                        if noisy_count.get(label, 0) < 2:
                            dataset.append((swell_with_harmonics_oscillations(t, alpha, 0.2, 0.5, alpha1, alpha3, alpha5, alpha7, beta, phi, 0.2, omega_n, noise=True), label, phi, True))
                            noisy_count[label] = noisy_count.get(label, 0) + 1
                        dataset.append((swell_with_harmonics_oscillations(t, alpha, 0.2, 0.5, alpha1, alpha3, alpha5, alpha7, beta, phi, 0.2, omega_n), label, phi, False))

    for alpha in np.linspace(0.1, 0.8, 5):
        for alpha1, alpha3, alpha5, alpha7 in zip(np.linspace(0.8, 1.2, 3), np.linspace(0.2, 0.5, 3), np.linspace(0.05, 0.1, 3), np.linspace(0.01, 0.05, 3)):
            for beta in np.linspace(0.2, 0.5, 3):
                for omega_n in np.linspace(2 * np.pi * 50, 2 * np.pi * 100, 5):
                    for phi in phases:
                        label = 'SagWithHarmonicsOscillations'
                        if noisy_count.get(label, 0) < 2:
                            dataset.append((sag_with_harmonics_oscillations(t, alpha, 0.2, 0.5, alpha1, alpha3, alpha5, alpha7, beta, phi, 0.2, omega_n, noise=True), label, phi, True))
                            noisy_count[label] = noisy_count.get(label, 0) + 1
                        dataset.append((sag_with_harmonics_oscillations(t, alpha, 0.2, 0.5, alpha1, alpha3, alpha5, alpha7, beta, phi, 0.2, omega_n), label, phi, False))
    return dataset


train_dir = 'output_new/train'
test_dir = 'output_new/test'

def ensure_min_samples_per_class(dataset, min_samples=2):
    unique_labels = set([item[1] for item in dataset]) 
    for label in unique_labels:
        count = sum(1 for item in dataset if item[1] == label)
        if count < min_samples:
            idx = next(i for i, item in enumerate(dataset) if item[1] == label)
            for _ in range(min_samples - count):
                dataset.append(dataset[idx])
    return dataset

dataset = generate_dataset()

dataset = ensure_min_samples_per_class(dataset)
X = [item[0] for item in dataset]
y = [item[1] for item in dataset]
phis = [item[2] for item in dataset]
noise = [item[3] if len(item) > 3 else False for item in dataset]

X_train, X_test, y_train, y_test, phi_train, phi_test, noise_train, noise_test = train_test_split(
    X, y, phis, noise, test_size=0.3, random_state=42, stratify=y
)

train_dir = 'output_new/train'
test_dir = 'output_new/test'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

def save_pq_event(data, label, phi, is_noisy, save_dir, index):
    base_path = os.path.join(save_dir, label)
    os.makedirs(base_path, exist_ok=True)
    
    save_path = os.path.join(base_path, f"{label}_WithoutNoise_{index}.png")
    print(save_path)
    plot_pq_event(lambda t, p: data, t, save_path, phi)
    
    if is_noisy:
        noisy_data = data + np.random.normal(0, 0.1, len(data))
        save_path = os.path.join(base_path, f"{label}_WithNoise_{index}.png")
        plot_pq_event(lambda t, p: noisy_data, t, save_path, phi)

for i, (data, label, phi, is_noisy) in enumerate(zip(X_train, y_train, phi_train, noise_train)):
    save_pq_event(data, label, phi, is_noisy, train_dir, i+1)

for i, (data, label, phi, is_noisy) in enumerate(zip(X_test, y_test, phi_test, noise_test)):
    save_pq_event(data, label, phi, is_noisy, test_dir, i+1)

print("Dataset generation and saving completed.")

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import os
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b6')
model._fc = torch.nn.Identity()
model.eval()

transform = transforms.Compose([
    transforms.RandomResizedCrop(600, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((600, 600)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def extract_features(image_path, model, transform):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        features = model(image)
    return features.squeeze().numpy()

train_dir = 'output_new/train'
test_dir = 'output_new/test'

all_train_features = []
all_train_features_labels = []

all_test_features = []
all_test_features_labels = []

for label in os.listdir(train_dir):
    label_dir = os.path.join(train_dir, label)
    if os.path.isdir(label_dir):
        for image_name in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_name)
            features = extract_features(image_path, model, transform)
            all_train_features.append(features)
            all_train_features_labels.append(os.listdir(test_dir).index(label))

for label in os.listdir(test_dir):
    label_dir = os.path.join(test_dir, label)
    if os.path.isdir(label_dir):
        for image_name in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_name)
            features = extract_features(image_path, model, test_transform)
            all_test_features.append(features)
            all_test_features_labels.append(os.listdir(test_dir).index(label))

all_train_features = np.array(all_train_features)
all_train_features_labels = np.array(all_train_features_labels)

all_test_features = np.array(all_test_features)
all_test_features_labels = np.array(all_test_features_labels)

print("Feature extraction completed.")
print("Train features shape:", all_train_features.shape)
print("Train labels shape:", all_train_features_labels.shape)
print("Test features shape:", all_test_features.shape)
print("Test labels shape:", all_test_features_labels.shape)

In [ ]:
import numpy as np
np.save("all_train_features.npy", all_train_features)
np.save("all_train_features_labels.npy", all_train_features_labels)
np.save("all_test_features.npy", all_test_features)
np.save("all_test_features_labels.npy", all_test_features_labels)